In [1]:
import rdflib
import pandas as pd

from tqdm import tqdm_notebook

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold

import sys
sys.path.append('..')
from tree_builder import KGPTree, KGPForest, KPGTransformer
from datastructures import *
import time

import pickle

import warnings; warnings.filterwarnings('ignore')

# Path Feature Transformation

In [2]:
rdf_file = 'data/AIFB/aifb.n3'
_format = 'n3'
train_file = 'data/AIFB/AIFB_test.tsv'
test_file = 'data/AIFB/AIFB_train.tsv'
entity_col = 'person'
label_col = 'label_affiliation'
label_predicates = [
    rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation'),
    rdflib.URIRef('http://swrc.ontoware.org/ontology#employs'),
    rdflib.URIRef('http://swrc.ontoware.org/ontology#carriedOutBy')
]
output = 'output/aifb_depth10.p'

In [3]:
print(end='Loading data... ', flush=True)
g = rdflib.Graph()
g.parse(rdf_file, format=_format)
print('OK')

test_data = pd.read_csv(train_file, sep='\t')
train_data = pd.read_csv(test_file, sep='\t')

train_entities = [rdflib.URIRef(x) for x in train_data[entity_col]]
train_labels = train_data[label_col]

test_entities = [rdflib.URIRef(x) for x in test_data[entity_col]]
test_labels = test_data[label_col]

kg = KnowledgeGraph.rdflib_to_kg(g, label_predicates=label_predicates)

Loading data... OK


In [4]:
transf = KPGTransformer(kg, path_max_depth=8, n_features=10000, progress=tqdm_notebook)
transf.fit(train_entities, train_labels)

In [5]:
train_features = transf.transform(train_entities)
test_features = transf.transform(test_entities)

useful_features = np.sum(train_features, axis=0) > 1

train_features = train_features[:, useful_features]
test_features = test_features[:, useful_features]

print(train_features.shape)


(140, 7973)


In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = GridSearchCV(RandomForestClassifier(max_features=None), 
                   {'n_estimators': [10, 100, 250, 500], 'max_depth': [3, 5, 10, None]})
clf.fit(train_features, train_labels)
print(clf.best_params_)
preds = clf.predict(test_features)
print(accuracy_score(test_labels, preds))

{'max_depth': 5, 'n_estimators': 250}
0.8888888888888888


In [7]:
from sklearn.linear_model import LogisticRegression

clf = GridSearchCV(LogisticRegression(penalty='l1'), {'C': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]})
clf.fit(train_features, train_labels)
print(clf.best_params_)
preds = clf.predict(test_features)
print(accuracy_score(test_labels, preds))

{'C': 1.0}
0.8888888888888888


In [8]:
from sklearn.svm import SVC

clf = GridSearchCV(SVC(), {'kernel': ['rbf', 'linear'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]})
clf.fit(train_features, train_labels)
print(clf.best_params_)
preds = clf.predict(test_features)
print(accuracy_score(test_labels, preds))

{'C': 100.0, 'kernel': 'rbf'}
0.8611111111111112


# Single KG Path Tree

In [9]:
print(end='Loading data... ', flush=True)
g = rdflib.Graph()
g.parse(rdf_file, format=_format)
print('OK')

test_data = pd.read_csv(train_file, sep='\t')
train_data = pd.read_csv(test_file, sep='\t')

train_entities = [rdflib.URIRef(x) for x in train_data[entity_col]]
train_labels = train_data[label_col]

test_entities = [rdflib.URIRef(x) for x in test_data[entity_col]]
test_labels = test_data[label_col]

kg = KnowledgeGraph.rdflib_to_kg(g, label_predicates=label_predicates)

clf = KGPTree(kg, path_max_depth=8, min_samples_leaf=1, max_tree_depth=None, progress=tqdm_notebook)

Loading data... OK


In [10]:
%%time
clf.fit(train_entities, train_labels)

CPU times: user 49.9 s, sys: 557 ms, total: 50.4 s
Wall time: 50.4 s


In [11]:
preds = clf.predict(test_entities)
print(accuracy_score(test_labels, preds))

0.9166666666666666


# Forest of KG Path Trees

In [12]:
print(end='Loading data... ', flush=True)
g = rdflib.Graph()
g.parse(rdf_file, format=_format)
print('OK')

test_data = pd.read_csv(train_file, sep='\t')
train_data = pd.read_csv(test_file, sep='\t')

train_entities = [rdflib.URIRef(x) for x in train_data[entity_col]]
train_labels = train_data[label_col]

test_entities = [rdflib.URIRef(x) for x in test_data[entity_col]]
test_labels = test_data[label_col]

kg = KnowledgeGraph.rdflib_to_kg(g, label_predicates=label_predicates)

clf = KGPForest(kg, path_max_depth=8, 
                min_samples_leaf=1, 
                max_tree_depth=None,
                n_estimators=10,
                vertex_sample=0.5,
                progress=tqdm_notebook,
                n_jobs=1)

# CPU times: user 2min 5s, sys: 2.09 s, total: 2min 7s
# Wall time: 2min 45s

Loading data... OK


In [13]:
%%time
clf.fit(train_entities, train_labels)


CPU times: user 2min 4s, sys: 2.18 s, total: 2min 6s
Wall time: 2min 5s


In [14]:
preds = clf.predict(test_entities)
print(accuracy_score(test_labels, preds))


0.8611111111111112
